In [ ]:
from google.colab import drive

drive.mount('/content/drive')

In [2]:
!mkdir -p /content/drive/My\ Drive/Colab\ Notebooks/DL-sentiment-analysis/checkpoints

In [3]:
!git clone https://github.com/mschroederi/DL-sentiment-analysis.git

Cloning into 'DL-sentiment-analysis'...
remote: Enumerating objects: 76, done.
remote: Counting objects: 100% (76/76), done.
remote: Compressing objects: 100% (66/66), done.
remote: Total 207 (delta 27), reused 49 (delta 7), pack-reused 131
Receiving objects: 100% (207/207), 106.96 MiB | 12.27 MiB/s, done.
Resolving deltas: 100% (80/80), done.
Checking out files: 100% (43/43), done.


In [4]:
!ls
%cd DL-sentiment-analysis/

DL-sentiment-analysis  drive  sample_data
/content/DL-sentiment-analysis


In [5]:
# train.py
import torch

from app.trainers.lstm_classifier_trainer import LSTMClassifierTrainer

if __name__ == '__main__':
    trainer = LSTMClassifierTrainer(use_grid_search=False)
    trainer.train(train_data_path='data/train.csv', 
                  model_checkpoint_path='/content/drive/My Drive/Colab Notebooks/DL-sentiment-analysis/checkpoints/model.pt', 
                  vocab_checkpoint_path='/content/drive/My Drive/Colab Notebooks/DL-sentiment-analysis/checkpoints/vocab.txt', 
                  num_epochs=50, patience=5)

Running on the GPU
Starting preprocessing pipeline...
Completed preprocessing pipeline.
Epoch: 1, Train Loss: 0.0026783743904529355, Train acc: 0.5566844381852932, Validation Loss: 0.0026480302970283707, Validation acc: 0.6163559831629585
Validation loss decreased from inf to 0.0026 in epoch 0.  Creating model checkpoint ...

Epoch: 2, Train Loss: 0.0024437821015735085, Train acc: 0.6476712604365757, Validation Loss: 0.00250805728921529, Validation acc: 0.6550410903988775
Validation loss decreased from 0.0026 to 0.0025 in epoch 1.  Creating model checkpoint ...

Epoch: 3, Train Loss: 0.0023547989953248962, Train acc: 0.6634644401971632, Validation Loss: 0.002398513300812156, Validation acc: 0.6760873922629785
Validation loss decreased from 0.0025 to 0.0024 in epoch 2.  Creating model checkpoint ...

Epoch: 4, Train Loss: 0.0021952182869953885, Train acc: 0.7196962076249874, Validation Loss: 0.002064695271539602, Validation acc: 0.7552615754660252
Validation loss decreased from 0.0024 t

In [6]:
# test.py
from app.testers.lstm_classifier_tester import LSTMClassifierTester

if __name__ == '__main__':
    tester = LSTMClassifierTester()
    tester.test(test_data_path='data/test.csv',
                model_checkpoint_path='/content/drive/My Drive/Colab Notebooks/DL-sentiment-analysis/checkpoints/model.pt', 
                vocab_path='/content/drive/My Drive/Colab Notebooks/DL-sentiment-analysis/checkpoints/vocab.txt')


Running on the GPU
Starting preprocessing pipeline...
Completed preprocessing pipeline.
Test Loss: 0.0012763278670538678, Test acc: 0.874251977991084


In [7]:
# write_review.py
import argparse
from tkinter import *

import numpy as np
import torch

from app.embeddings.sequence_tokenizer import SequenceTokenizer
from app.models.lstm_classifier import LSTMClassifier
from app.preprocessing.preprocessor import Preprocessor

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

model: LSTMClassifier = torch.load('/content/drive/My Drive/Colab Notebooks/DL-sentiment-analysis/checkpoints/model.pt', map_location=device)
padding_size = model.padding_size
tokenizer = SequenceTokenizer.from_vocab_file('/content/drive/My Drive/Colab Notebooks/DL-sentiment-analysis/checkpoints/vocab.txt', padding_size)
model.eval()
print('Successfully loaded the model.')

Successfully loaded the model.


In [8]:
# write_review.py

review = 'This was the best movie I\'ve ever watched. I would recommend it to all my friends.'


preprocessed = Preprocessor.preprocess_text(review)
tokenized = tokenizer._SequenceTokenizer__transform_single_review(preprocessed)

with torch.no_grad():
    X = torch.tensor(tokenized).reshape(-1, padding_size).to(device)
    y = model(X).reshape(-1, 1)
    prob = y.flatten().item()
percentage = np.round(prob * 100, 2)

print("Positive Review: {}%".format(percentage))

Positive Review: 94.89%


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  # Remove the CWD from sys.path while we load stuff.
